In [31]:
## Enter your athletic.net log in information ##

my_email = 'chad_r_smith@charter.net'
my_password = 'maudMenten1!'

## ------------------------------------------ ##

In [32]:
import csv
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [33]:
class_dict = {145512:"A", 145517:"B", 145524:"C", 145534:"D"}
genders_dict = {"m":"Boys", "f":"Girls"}
classes = [145512, 145517, 145524, 145534]
genders = ["m", 'f']

In [34]:
# Launch a browser and website 
driver = webdriver.Chrome()
driver.get('https://www.athletic.net/TrackAndField/rankings/list/145512/m?depth=25')

# Find the log in button by its class name
buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "a-btn.athletic-header--btn.ng-star-inserted")))

buttons[1].click()

# input email and password into fields
password_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Enter Password"][formcontrolname="password"]')))
email_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Enter Email"][formcontrolname="email"]')))

email_input.send_keys(my_email)
password_input.send_keys(my_password)

# click button to submit log in 
login_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))

login_button.click()

# pause to allow log in to compete
time.sleep(5)

# get page content
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')


In [35]:
headers = ['Event', 'rank', 'profile', 'Athlete', 'blank_1', 'Mark', 'blank_2', 'ST', 'Age', 'Grade', 'blank_3', 'Team', 'Date', 'Meet']

g_class_dict = {}
agg_df = pd.DataFrame(columns=["Event", "rank", "Age", "Grade", "Athlete", "Mark", "ST", "Team", "Date", "Meet"])

for g in genders:
    for class_ in classes:
        
        url = f'https://www.athletic.net/TrackAndField/rankings/list/{class_}/{g}?depth=25'
        driver.get(url)
        
        time.sleep(4)

        # get page content
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')


        data = [headers]
        rows = soup.find_all('tr')

        for i, row in enumerate(rows):  # Skip the first row (headers)
            row_list = []

            cells = row.find_all(['td', 'th'])  # Extract data from each cell in the row

            for header, cell in zip(headers, cells):
                cell_contents = str(cell.get_text().strip().replace('*', ''))
                row_list.append(cell_contents)

            if 'Gr' not in row_list:
                data.append(row_list)


        df = pd.DataFrame(data)

        df.columns = df.loc[0, :]
        df = df.drop(0).reset_index(drop=True)
        df.drop(columns=['blank_1', 'blank_2', 'blank_3', 'profile'], inplace=True)
        df['rank'] = df['rank'].apply(lambda x: x.replace('.', ''))

        df['Event'] = df['Event'].apply(lambda x: x[:-15].rstrip())
        df = df[["Event", "rank", "Age", "Grade", "Athlete", "Mark", "ST", "Team", "Date", "Meet"]]

        class_g_label = genders_dict[g] + str(class_dict[class_])
        print(class_g_label, len(df))
        
        agg_df = pd.concat([agg_df, df], ignore_index=True)
        
        g_class_dict[class_g_label] = df

        time.sleep(2)

    
# Close the WebDriver
driver.quit()


import datetime as dt
curr_date = dt.date.today().strftime('%m_%d')
agg_df.to_csv(f'{curr_date}_Track_and_Field_leaderboard.csv', header=False, index=False)


BoysA 451
BoysB 466
BoysC 467
BoysD 475
GirlsA 453
GirlsB 475
GirlsC 483
GirlsD 470


In [36]:
import datetime as dt
curr_date = dt.date.today().strftime('%m_%d')


with pd.ExcelWriter(f'prep_running_nerd/{curr_date}_Track_and_Field_leaderboard.xlsx', engine='xlsxwriter') as writer:
    agg_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    for div_name in g_class_dict.keys():
        sheet_name = div_name      
        g_class_dict[div_name].to_excel(writer, sheet_name=sheet_name, index=False)

# g_dict['f']
g_class_dict.keys()

dict_keys(['BoysA', 'BoysB', 'BoysC', 'BoysD', 'GirlsA', 'GirlsB', 'GirlsC', 'GirlsD'])